<a href="https://colab.research.google.com/github/MartinVIllesca/CC6205-Procesamiento-de-Lenguaje-Natural/blob/master/Tarea2/Tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 2 - Named Entity Recognition

----------------------

- **Nombre:** Martín Valderrama, Jou-Hui Ho

- **Usuario o nombre de equipo en Codalab:** pibjou



## Introducción a la tarea

### Objetivo


El objetivo de esta tarea es resolver una de las tasks mas importantes de Sequence Labelling: [Named Entity Recognition (NER)](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf). 


Esperamos que (por lo menos) utilizen Redes Neuronales Recurrentes (RNN) para resolverla. Nuevamente, hay total libertad para utilizar software y los modelos que deseen, siempre y cuando estos no traigan los modelos ya implementados (como el caso de spacy).


**Named Entity Recognition (NER)**

Esta tarea consiste en localizar y clasificar los tokens de una oración que representen entidades nombradas. Es decir, tokens que simbolicen (1) **personas**, (2) **organizaciones**, (3) **lugares** y (4) **adjetivos, eventos y otras entidades que no entren en las categorías anteriores** deberán ser taggeados como (1) **PER**, (2) **ORG**, (3) **LOC** y (4) **MISC** respectivamente. Adicionalmente, dado que existen entidades representadas en más de un token (como La Serena), se utiliza la notación BIO como prefijo al tag: Beginning, Inside, Outside. Es decir, si encuentro una entidad, el primer token etiquetado será precedido por B, el segundo por I y los n restantes por I. Por otra parte, si el token no representa ninguna entidad nombrada, se representa por O. Un ejemplo de esto es:

Por ejemplo:

```
Felipe B-PER
Bravo I-PER
es O
el O
profesor O
de O
PLN B-MISC
de O
la O
Universidad B-ORG
de I-ORG
Chile I-ORG
. O
```

Estos links son los más indicados para comenzar:

-  [Tagging, and Hidden Markov Models ](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf) (slides by Michael Collins), [notes](http://www.cs.columbia.edu/~mcollins/hmms-spring2013.pdf), [video 1](https://youtu.be/-ngfOZz8yK0), [video 2](https://youtu.be/PLoLKQwkONw), [video 3](https://youtu.be/aaa5Qoi8Vco), [video 4](https://youtu.be/4pKWIDkF_6Y)       
-  [Recurrent Neural Networks](slides/NLP-RNN.pdf) | [video 1](https://youtu.be/BmhjUkzz3nk), [video 2](https://youtu.be/z43YFR1iIvk), [video 3](https://youtu.be/7L5JxQdwNJk)


Recuerden que todo el material se encuentra disponible en el [github del curso](https://github.com/dccuchile/CC6205).

**Link a la competencia:  https://competitions.codalab.org/competitions/25302?secret_key=690406c7-b3b0-4092-8694-d08d7991ca94**

### Reporte

Este debe cumplir la siguiente estructura:

1.	**Introducción**: Presentar brevemente el problema a resolver, los modelos utilizados en el desarrollo de la tarea y conclusiones obtenidas. (0.5 Puntos)

2.	**Modelos**: Describir brevemente los modelos, métodos y hiperparámetros utilizados. (1.0 puntos)

4.	**Métricas de evaluación**: Describir las métricas utilizadas en la evaluación indicando que miden y cuál es su interpretación en este problema en particular. (0.5 puntos)

5.	**Experimentos**: Reportar todos sus experimentos y código en esta sección. Comparar los resultados obtenidos utilizando diferentes modelos. ¡Es vital haber realizado varios experimentos para sacar una buena nota! (3.0 puntos)

6.	**Conclusiones**: Discutir resultados, proponer trabajo futuro. (1.0 punto)


-----------------------------------------

## Introducción


...

## Modelos 


...

## Métricas de evaluación

- **Precision:** ...
- **Recall:** ...
- **F1 score:** ...

## Experimentos


###  Carga de datos y Preprocesamiento

El proceso será el siguiente: 

1. Descargar los datos desde github y examinarlos.
2. Definir los campos (`fields`) que cargaremos desde los archivos.
3. Cargar los datasets.
4. Crear el vocabulario.



In [1]:
# Instalar torchtext (en codalab) - Descomentar.
!pip3 install --upgrade torchtext

     |████████████████████████████████| 71kB 3.1MB/s 
     |████████████████████████████████| 1.1MB 8.5MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [2]:
import torch
from torchtext import data, datasets

# Garantizar reproducibilidad 
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
%%capture
!wget https://github.com/dccuchile/CC6205/releases/download/Data/train_NER_esp.txt -nc # Dataset de Entrenamiento
!wget https://github.com/dccuchile/CC6205/releases/download/Data/val_NER_esp.txt -nc    # Dataset de Validación (Para probar y ajustar el modelo)
!wget https://github.com/dccuchile/CC6205/releases/download/Data/test_NER_esp.txt -nc  # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!

####  Fields

Un `field`:

* Define un tipo de datos junto con instrucciones para convertir el texto a Tensor.
* Contiene un objeto `Vocab` que contiene el vocabulario (palabras posibles que puede tomar ese campo).
* Contiene otros parámetros relacionados con la forma en que se debe numericalizar un tipo de datos, como un método de tokenización y el tipo de Tensor que se debe producir.


Analizemos el siguiente cuadro el cual contiene un ejemplo cualquiera de entrenamiento:


```
El O
Abogado B-PER
General I-PER
del I-PER
Estado I-PER
, O
Daryl B-PER
Williams I-PER
```

Cada linea contiene una palabra y su clase. Para que `torchtext` pueda cargar estos datos, debemos definir como va a leer y separar los componentes de cada una de las lineas.
Para esto, definiremos un field para cada uno de esos componentes: Las palabras (`TEXT`) y los NER_TAGS (`clase`).


In [4]:
# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = data.Field(lower=False) 

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = data.Field(unk_token=None)

fields = (("text", TEXT), ("nertags", NER_TAGS))

####  SequenceTaggingDataset

`SequenceTaggingDataset` es una clase de torchtext diseñada para contener datasets de sequence labelling. 
Los ejemplos que se guarden en una instancia de estos serán arreglos de palabras pareados con sus respectivos tags.
Por ejemplo, para Part-of-speech tagging:

[I, love, PyTorch, .] estará pareado con [PRON, VERB, PROPN, PUNCT]


La idea es que usando los fields que definimos antes, le indiquemos a la clase cómo cargar los datasets de prueba, validación y test.

In [5]:
train_data, valid_data, test_data = datasets.SequenceTaggingDataset.splits(
    path="./",
    train="train_NER_esp.txt",
    validation="val_NER_esp.txt",
    test="test_NER_esp.txt",
    fields=fields,
    encoding="iso-8859-1",
    separator=" "
)

print(f"Numero de ejemplos de entrenamiento: {len(train_data)}")
print(f"Número de ejemplos de validación: {len(valid_data)}")
print(f"Número de ejemplos de test (competencia): {len(test_data)}")

Numero de ejemplos de entrenamiento: 8323
Número de ejemplos de validación: 1915
Número de ejemplos de test (competencia): 1517


In [6]:
# # Un ejemplo
import random
random_item_idx = random.randint(0, len(train_data))
random_example = train_data.examples[random_item_idx]
list(zip(random_example.text, random_example.nertags))

[('Wolff', 'B-PER'),
 (',', 'O'),
 ('ahora', 'O'),
 ('periodista', 'O'),
 ('en', 'O'),
 ('Argentina', 'B-LOC'),
 (',', 'O'),
 ('jugó', 'O'),
 ('con', 'O'),
 ('Del', 'B-PER'),
 ('Bosque', 'I-PER'),
 ('en', 'O'),
 ('los', 'O'),
 ('últimos', 'O'),
 ('años', 'O'),
 ('de', 'O'),
 ('los', 'O'),
 ('70', 'O'),
 ('en', 'O'),
 ('el', 'O'),
 ('Real', 'B-ORG'),
 ('Madrid', 'I-ORG'),
 ('.', 'O')]

#### Construir los vocabularios para el texto y las etiquetas

Los vocabularios son los obbjetos que contienen todos los tokens (de entrenamiento) posibles para ambos fields.
El siguiente paso consiste en construirlos. Para esto, hacemos uso del método `Field.build_vocab` sobre cada uno de nuestros `fields`. 

In [7]:
TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data)

print(f"Tokens únicos en TEXT: {len(TEXT.vocab)}")
print(f"Tokens únicos en NER_TAGS: {len(NER_TAGS.vocab)}")

#Veamos las posibles etiquetas que hemos cargado:
print(NER_TAGS.vocab.itos)

# Tokens mas frecuentes
TEXT.vocab.freqs.most_common(10)

Tokens únicos en TEXT: 26101
Tokens únicos en NER_TAGS: 10
['<pad>', 'O', 'B-ORG', 'I-ORG', 'B-LOC', 'B-PER', 'I-PER', 'I-MISC', 'B-MISC', 'I-LOC']


[('de', 17657),
 (',', 14716),
 ('la', 9571),
 ('que', 7516),
 ('.', 7263),
 ('el', 6905),
 ('en', 6484),
 ('"', 5691),
 ('y', 5336),
 ('a', 4304)]

In [8]:
# Seteamos algunas variables que nos serán de utilidad mas adelante...
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']

In [9]:
# Frecuencia de los Tags

def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages

print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag Ocurrencia Porcentaje

O	231920	87.6%
B-ORG	7390	 2.8%
I-ORG	4992	 1.9%
B-LOC	4913	 1.9%
B-PER	4321	 1.6%
I-PER	3903	 1.5%
I-MISC	3212	 1.2%
B-MISC	2173	 0.8%
I-LOC	1891	 0.7%


#### Configuramos pytorch y dividimos los datos.


In [10]:
BATCH_SIZE = 16  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


### Helper functions

In [11]:
import time

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    for batch in iterator:
        text = batch.text
        tags = batch.nertags

        optimizer.zero_grad()

        #text = [sent len, batch size]

        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]

        loss = criterion(predictions, tags)
        precision, recall, f1 = calculate_metrics(predictions, tags)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)


def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            tags = batch.nertags

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)
            precision, recall, f1 = calculate_metrics(predictions, tags)

            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [12]:
def run_training(model, train_iterator, valid_iterator, optimizer, criterion, 
                 n_epochs, patience=2):
  best_valid_loss = float('inf')
  prev_loss = float('inf')

  # Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
  model = model.to(device)
  criterion = criterion.to(device)

  counter = 0
  for epoch in range(n_epochs):

      start_time = time.time()
      
      train_loss, train_precision, train_recall, train_f1 = train(
          model, train_iterator, optimizer, criterion)

      valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
          model, valid_iterator, criterion)

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
      # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), '{}.pt'.format(model_name))
      # Si ya no mejoramos el loss de validación, terminamos de entrenar.

      print(f'Epoch: {epoch+1:02}/{n_epochs} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(
          f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
      )
      print(
          f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
      )

      # Early stopping
      if valid_loss >= prev_loss:
        counter += 1
      
      if counter == patience:
        break

      prev_loss = valid_loss
  
  
  # cargar el mejor modelo entrenado.
  model.load_state_dict(torch.load('{}.pt'.format(model_name)))

  # Evaluamos el set de validación con el modelo final
  valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
      model, valid_iterator, criterion)
  print('------------------------------------------------')
  print('---------------- BEST MODEL --------------------')
  print('------------------------------------------------')
  print(
      f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
  )
  

In [13]:
# Definimos las métricas

# Noten que la evaluación solo se hace para las Named Entities (sin contar 'O').

from sklearn.metrics import f1_score, precision_score, recall_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore",
                        category=sklearn.exceptions.UndefinedMetricWarning)


def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)
    # Obtenemos los indices distintos de 0.

    # filtramos <pad> y O para calcular los scores.
    mask = [(y_true != o_idx) & (y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu')
    y_true = y_true.to('cpu')
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')

    return precision, recall, f1

-------------------

### Modelo Baseline

Teniendo ya cargado los datos, toca definir nuestro modelo. Este baseline tendrá una capa de embedding, unas cuantas LSTM y una capa de salida y usará dropout en el entrenamiento.

Este constará de los siguientes pasos: 

1. Definir la clase que contendrá la red.
2. Definir los hiperparámetros e inicializar la red. 
3. Definir la época de entrenamiento
3. Definir la función de loss.



Recomendamos que para experimentar, encapsules los modelos en una sola variable y luego la fijes en model para entrenarla

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx)

        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

#### Hiperparámetros de la red


In [15]:
def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


In [16]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100  # dimensión de los embeddings.
HIDDEN_DIM = 128  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 2  # número de capas.
DROPOUT = 0.25
BIDIRECTIONAL = False

# Creamos nuestro modelo.
baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...
baseline_n_epochs = 10

# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [17]:
model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs

optimizer = optim.Adam(model.parameters())

model.apply(init_weights)

NER_RNN(
  (embedding): Embedding(26101, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=2, dropout=0.25)
  (fc): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

Por último, definimos los embeddings que representan a \<unk\> y \<pad\>  como [0, 0, ..., 0]

In [19]:
run_training(model, train_iterator, valid_iterator, optimizer, criterion, n_epochs=10)

Epoch: 01/10 | Epoch Time: 0m 6s
	Train Loss: 0.446 | Train f1: 0.11 | Train precision: 0.14 | Train recall: 0.12
	 Val. Loss: 0.322 |  Val. f1: 0.26 |  Val. precision: 0.30 | Val. recall: 0.27
Epoch: 02/10 | Epoch Time: 0m 6s
	Train Loss: 0.184 | Train f1: 0.44 | Train precision: 0.49 | Train recall: 0.44
	 Val. Loss: 0.235 |  Val. f1: 0.47 |  Val. precision: 0.56 | Val. recall: 0.46
Epoch: 03/10 | Epoch Time: 0m 6s
	Train Loss: 0.114 | Train f1: 0.62 | Train precision: 0.67 | Train recall: 0.63
	 Val. Loss: 0.235 |  Val. f1: 0.52 |  Val. precision: 0.60 | Val. recall: 0.51
Epoch: 04/10 | Epoch Time: 0m 6s
	Train Loss: 0.078 | Train f1: 0.72 | Train precision: 0.75 | Train recall: 0.72
	 Val. Loss: 0.228 |  Val. f1: 0.57 |  Val. precision: 0.64 | Val. recall: 0.58
Epoch: 05/10 | Epoch Time: 0m 6s
	Train Loss: 0.059 | Train f1: 0.78 | Train precision: 0.80 | Train recall: 0.78
	 Val. Loss: 0.234 |  Val. f1: 0.57 |  Val. precision: 0.64 | Val. recall: 0.57
Epoch: 06/10 | Epoch Time: 0m 

### Modelos

*   **[DONE]** Probar Early stopping
*   Variar la cantidad de parámetros de la capa de embeddings.
*   **[DONE]** Variar la cantidad de capas RNN.
*   Variar la cantidad de parámetros de las capas de RNN.
*   Inicializar la capa de embeddings con modelos pre-entrenados. (word2vec, glove, conceptnet, etc...).[Guía breve aquí](https://github.com/dccuchile/spanish-word-embeddings), [Embeddings en español aquí](https://github.com/dccuchile/spanish-word-embeddings).
*   **[DONE]** Variar la cantidad de épocas de entrenamiento.
*   Variar el optimizador, learning rate, batch size, usar CRF loss, etc...
*   **[DONE]** Probar bi-direccionalidad.
*   Probar teacher forcing.
*   **[DONE]** Incluir dropout.
*   Probar modelos de tipo GRU
*   Probar Embedding Contextuales (les puede ser de utilidad [flair](https://github.com/flairNLP/flair))
*   Probar modelos de transformers en español usando [Huggingface](https://github.com/huggingface/transformers)

--------------------
### Modelo 1


In [21]:
BIDIRECTIONAL = True
model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX).apply(init_weights)

model_name = 'bi_rnn'  # nombre que tendrá el modelo guardado
n_epochs = 10

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
optimizer = optim.Adam(model.parameters())

run_training(model, train_iterator, valid_iterator, optimizer, criterion, n_epochs)

Epoch: 01/10 | Epoch Time: 0m 10s
	Train Loss: 0.359 | Train f1: 0.25 | Train precision: 0.32 | Train recall: 0.24
	 Val. Loss: 0.289 |  Val. f1: 0.43 |  Val. precision: 0.53 | Val. recall: 0.41
Epoch: 02/10 | Epoch Time: 0m 10s
	Train Loss: 0.125 | Train f1: 0.60 | Train precision: 0.66 | Train recall: 0.60
	 Val. Loss: 0.270 |  Val. f1: 0.55 |  Val. precision: 0.65 | Val. recall: 0.53
Epoch: 03/10 | Epoch Time: 0m 10s
	Train Loss: 0.070 | Train f1: 0.74 | Train precision: 0.77 | Train recall: 0.74
	 Val. Loss: 0.281 |  Val. f1: 0.56 |  Val. precision: 0.65 | Val. recall: 0.54
Epoch: 04/10 | Epoch Time: 0m 10s
	Train Loss: 0.043 | Train f1: 0.82 | Train precision: 0.84 | Train recall: 0.82
	 Val. Loss: 0.292 |  Val. f1: 0.58 |  Val. precision: 0.67 | Val. recall: 0.55
------------------------------------------------
---------------- BEST MODEL --------------------
------------------------------------------------
Val. Loss: 0.270 |  Val. f1: 0.55 | Val. precision: 0.65 | Val. recall: 0

---------------

### Modelo 2

In [22]:
model = NER_RNN(input_dim=len(TEXT.vocab),
                embedding_dim=100,
                hidden_dim=200,
                output_dim=len(NER_TAGS.vocab),
                n_layers=3,
                bidirectional=True,
                dropout=0.2,
                pad_idx=PAD_IDX).apply(init_weights)

print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

model_name = 'bigger_bi_rnn'  # nombre que tendrá el modelo guardado
n_epochs = 10

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=0.01)

run_training(model, train_iterator, valid_iterator, optimizer, criterion, n_epochs)

El modelo actual tiene 5,023,710 parámetros entrenables.
Epoch: 01/10 | Epoch Time: 0m 16s
	Train Loss: 0.326 | Train f1: 0.27 | Train precision: 0.31 | Train recall: 0.28
	 Val. Loss: 0.263 |  Val. f1: 0.41 |  Val. precision: 0.49 | Val. recall: 0.41
Epoch: 02/10 | Epoch Time: 0m 15s
	Train Loss: 0.126 | Train f1: 0.60 | Train precision: 0.65 | Train recall: 0.61
	 Val. Loss: 0.220 |  Val. f1: 0.54 |  Val. precision: 0.61 | Val. recall: 0.55
Epoch: 03/10 | Epoch Time: 0m 15s
	Train Loss: 0.076 | Train f1: 0.74 | Train precision: 0.77 | Train recall: 0.75
	 Val. Loss: 0.229 |  Val. f1: 0.57 |  Val. precision: 0.63 | Val. recall: 0.58
Epoch: 04/10 | Epoch Time: 0m 15s
	Train Loss: 0.056 | Train f1: 0.81 | Train precision: 0.83 | Train recall: 0.81
	 Val. Loss: 0.234 |  Val. f1: 0.59 |  Val. precision: 0.66 | Val. recall: 0.59
------------------------------------------------
---------------- BEST MODEL --------------------
------------------------------------------------
Val. Loss: 0.220

In [23]:
# glove pretrained embeddings
!wget http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz

--2020-07-22 03:29:21--  http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz
Resolving dcc.uchile.cl (dcc.uchile.cl)... 192.80.24.11
Connecting to dcc.uchile.cl (dcc.uchile.cl)|192.80.24.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz [following]
--2020-07-22 03:29:21--  https://www.dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz
Resolving www.dcc.uchile.cl (www.dcc.uchile.cl)... 200.9.99.213, 192.80.24.11
Connecting to www.dcc.uchile.cl (www.dcc.uchile.cl)|200.9.99.213|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://users.dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz [following]
--2020-07-22 03:29:22--  https://users.dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (u

---------------


### Modelo 3: Mayor batch size

In [24]:
BATCH_SIZE = 64  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


In [25]:
model = NER_RNN(input_dim=len(TEXT.vocab),
                embedding_dim=100,
                hidden_dim=200,
                output_dim=len(NER_TAGS.vocab),
                n_layers=3,
                bidirectional=True,
                dropout=0.3,
                pad_idx=PAD_IDX).apply(init_weights)

print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

model_name = 'bigger_bi_rnn'  # nombre que tendrá el modelo guardado
n_epochs = 10

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=0.01)

run_training(model, train_iterator, valid_iterator, optimizer, criterion, n_epochs)

El modelo actual tiene 5,023,710 parámetros entrenables.
Epoch: 01/10 | Epoch Time: 0m 8s
	Train Loss: 0.615 | Train f1: 0.01 | Train precision: 0.02 | Train recall: 0.01
	 Val. Loss: 0.420 |  Val. f1: 0.04 |  Val. precision: 0.06 | Val. recall: 0.08
Epoch: 02/10 | Epoch Time: 0m 8s
	Train Loss: 0.250 | Train f1: 0.20 | Train precision: 0.23 | Train recall: 0.23
	 Val. Loss: 0.314 |  Val. f1: 0.25 |  Val. precision: 0.30 | Val. recall: 0.26
Epoch: 03/10 | Epoch Time: 0m 8s
	Train Loss: 0.197 | Train f1: 0.32 | Train precision: 0.38 | Train recall: 0.35
	 Val. Loss: 0.318 |  Val. f1: 0.35 |  Val. precision: 0.47 | Val. recall: 0.35
Epoch: 04/10 | Epoch Time: 0m 8s
	Train Loss: 0.135 | Train f1: 0.52 | Train precision: 0.58 | Train recall: 0.53
	 Val. Loss: 0.279 |  Val. f1: 0.52 |  Val. precision: 0.57 | Val. recall: 0.54
Epoch: 05/10 | Epoch Time: 0m 8s
	Train Loss: 0.093 | Train f1: 0.67 | Train precision: 0.70 | Train recall: 0.67
	 Val. Loss: 0.273 |  Val. f1: 0.53 |  Val. precision

---------------


### Modelo 4: Agregar más capas

In [26]:
class NER_RNN2(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx)

        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)
        
        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Sequential(nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                                          hidden_dim),
                                nn.BatchNorm1d(num_features=hidden_dim),
                                nn.ReLU(),
                                nn.Linear(hidden_dim, output_dim))


    def forward(self, text):

        #text = [sent len, batch size]

        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)

        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [28]:
model = NER_RNN2(input_dim=len(TEXT.vocab),
                embedding_dim=100,
                hidden_dim=200,
                output_dim=len(NER_TAGS.vocab),
                n_layers=3,
                bidirectional=True,
                dropout=0.3,
                pad_idx=PAD_IDX).apply(init_weights)

print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

model_name = 'bigger_bi_rnn'  # nombre que tendrá el modelo guardado
n_epochs = 10

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=0.01)

run_training(model, train_iterator, valid_iterator, optimizer, criterion, n_epochs)

El modelo actual tiene 5,102,310 parámetros entrenables.


RuntimeError: ignored

In [ ]:
# Limpiar ram de cuda
torch.cuda.empty_cache()


## Predecir datos para la competencia


In [ ]:
def predict_labels(model, iterator, criterion, fields=fields):

    # Extraemos los vocabularios.
    text_field = fields[0][1]
    nertags_field = fields[1][1]
    tags_vocab = nertags_field.vocab.itos
    words_vocab = text_field.vocab.itos

    model.eval()

    predictions = []

    with torch.no_grad():

        for batch in iterator:

            text_batch = batch.text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(batch.text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,
                                                     predictions_batch):
                for word_idx, word_predictions in zip(sentence,
                                                      sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]

                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]

                    if current_word != '<pad>':
                        predictions.append([current_word, current_tag])


    return predictions


predictions = predict_labels(model, test_iterator, criterion)

### Generar el archivo para la submission

No hay problema si aparecen unk en la salida. Estos no son relevantes para evaluarlos, usamos solo los tags.

In [ ]:
import os, shutil

if (os.path.isfile('./predictions.zip')):
    os.remove('./predictions.zip')

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt', 'w')
for word, tag in predictions:
    f.write(word + ' ' + tag + '\n')
f.write('\n')
f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')

In [ ]:
# A veces no funciona a la primera. Ejecutar mas de una vez para obtener el archivo...
from google.colab import files
files.download('predictions.zip')  

## Conclusiones



...